In [68]:
from ultralytics import YOLO

model_name = 'yolov8x-pose'
YOLO(model_name)

YOLO(
  (model): PoseModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_runnin

In [69]:
!tar -zcvf {model_name}.tar.gz {model_name}.pt

a yolov8x-pose.pt


In [70]:
import sagemaker

sess = sagemaker.Session()

model_data = sess.upload_data(
    path=f"/Users/midpare/Documents/git/yolo_in_sagemaker/{model_name}.tar.gz", bucket='sagemaker-yolo-midpare', key_prefix='model'
)

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


In [71]:
from sagemaker.pytorch import PyTorchModel

role = 'arn:aws:iam::972511216533:role/sagemaker_role'

model = PyTorchModel(
    entry_point="inference.py",
    source_dir="code",
    role=role,
    model_data=model_data,
    env= {'MODEL': model_name + '.pt'},
    framework_version="1.12.1",
    py_version="py38",
)

In [73]:
from sagemaker.deserializers import JSONDeserializer

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.2xlarge',
    deserializer=JSONDeserializer(),
    endpoint_name='pytorch-inference-' + model_name
)

-----------------!